http://34.68.85.234:7860/

http://ec2-44-217-44-154.compute-1.amazonaws.com:7860/

In [1]:
import requests
import xml.etree.ElementTree as ET
from transformers import BartTokenizer, BartForConditionalGeneration
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import nltk
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize
nltk.download('punkt')  # Download the 'punkt' resource
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ritvikkhandelwal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ritvikkhandelwal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
from src.Model_Parameter import get_model_response

import re
import os
import csv
import json
from typing import List
import yaml
import openai
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm, trange
from langchain import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser

In [23]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ritvikkhandelwal/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
response = get_model_response("Hello")[0][1]
print(response)

Hi there! How can I help you today?


In [44]:
#create a prompt template

template = (
    """
    Please summarize the below text
    
    {context}
    """
)
prompt_template = PromptTemplate.from_template(template)
print(prompt_template.input_variables)

['context']


In [45]:
import os
import re

# Define the paths to the directories
text_dir = '/Users/ritvikkhandelwal/Desktop/Capstone/sumpubmed-master/Text final'
abstract_dir = '/Users/ritvikkhandelwal/Desktop/Capstone/sumpubmed-master/abstract final'
shorter_abstract_dir = '/Users/ritvikkhandelwal/Desktop/Capstone/sumpubmed-master/shorter_abstract_final'

def advanced_clean_text(text):
    # Convert text to lowercase for uniformity
    cleaned_text = text.lower()

    # Remove all tags formatted like <anything>
    cleaned_text = re.sub(r'<[^>]+>', '', cleaned_text)

    # Remove standalone numbers (which might be irrelevant like page numbers)
    cleaned_text = re.sub(r'\b\d+\b', '', cleaned_text)

    # Remove special characters, keeping alphabets, numbers, and basic punctuation
    cleaned_text = re.sub(r'[^a-z0-9\s\.\,\;\:]', '', cleaned_text)

    # Replace multiple spaces with a single space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    # Add a newline character after each period to segment sentences
    cleaned_text = re.sub(r'\.\s*', '.\n', cleaned_text)

    return cleaned_text

def read_files_from_directory(directory):
    #file_stats = []
    #vocabulary = Counter()
    contents_list = []
    
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                # Clean the text using the advanced cleaning function
                cleaned_content = advanced_clean_text(content)
                contents_list.append(cleaned_content)
                
    return contents_list

# Create lists for each type of file
text_list = read_files_from_directory(text_dir)
abstract_list = read_files_from_directory(abstract_dir)
short_abstract_list = read_files_from_directory(shorter_abstract_dir)

# The first element of each list would contain the contents of 'text_1.txt', 'abstract_1.txt', and 'abst_1.txt', respectively

In [46]:
def chunk_text(text, chunk_size):
    words = text.split()
    chunks = [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

def summarize_text(text, chunk_size=500):
    chunks = chunk_text(text, chunk_size)
    summarized_chunks = []
    for chunk in chunks:
        prompt = prompt_template.format(context=chunk)
        summary = get_model_response(prompt)[0][1]
        summarized_chunks.append(summary)

    return ' '.join(summarized_chunks)

In [47]:
text_list = text_list[0:10]
len(text_list)

10

In [49]:
# Generate summaries
import time

# Start time
start_time = time.time()

reference_summaries = [summarize_text(text) for text in text_list]

end_time = time.time()

time_taken = end_time - start_time
print(f"Time taken: {time_taken} seconds")

Time taken: 836.3300080299377 seconds


In [51]:
reference_summaries[0]

'The article discusses challenges associated with traditional cloning methods, particularly when working with larger inserts, and outlines alternatives that can simplify the process. These include the Gateway cloning system, which utilizes a proprietary protein formulation to catalyze sitespecific recombination in vitro; the Domino method and DNA Assembler, which rely on homologous recombination in vivo using the recombination machinery of certain bacteria (Bacillus subtilis or Saccharomyces cerevisiae); and the endogenous recombination system of Escherichia coli, which can combine insert and vector molecules through cotransfection, facilitated by the expression of a homing endonuclease and bacteriophage recombinases. Additionally, PCR-based methods can generate complete recombinant plasmids, including the use of proofreading DNA polymerase, and various strategies can be employed for genome engineering. Finally, Ligation-independent cloning techniques involve creating longer overhangs 

In [48]:
# rough

#output_parser = CommaSeparatedListOutputParser()

#format_instructions = output_parser.get_format_instructions()

prompt = prompt_template.format(
    context=text_list[0]
    #format_instructions = format_instructions
)

print(prompt)

output = get_model_response(prompt)[0][1]

print(output)


    Please summarize the below text
    
    background with hundreds of enzymes commercially available today , restriction endonuclease treatment of insert and plasmid vector dna followed by ligation and transformation into competent e.
coli strains presents the standard cloning method in molecular biology.
given the advances in structural biology and the advent of synthetic biology, a strong demand exists to transfer and rearrange a large variety of dna fragments from different genetic sources in a directed manner.
a diverse catalogue of plasmid vectors is at hand for propagation in pro and eukaryotic cells, enabling heterologous protein expression in various host organisms.
frequently, suitable pairs of type ii restriction enzymes with unique recognition sites in the vector and insert dna fragments can be found, especially since the latter are easily produced via pcr.
in such a case, the pcr primers contain addon tails composed of the restriction endonuclease recognition sequence a

The response provides a detailed explanation of how the proposed cloning method works and highlights its advantages over traditional cloning methods. The author explains that the method uses a combination of Type II restriction endonucleases and DNA polymerase to generate a cohesive terminus for the insert DNA sequence, which can then be ligated to a linearized vector. The method does not require the internalization of the insert sequence into the vector, which simplifies the process and reduces the risk of introducing unwanted mutations. Additionally, the use of shorter PCR primers reduces the risk of secondary structure formation and synthesis errors.
The response also discusses the potential applications of the method, including cloning of individual mutants in the context of libraries, and notes that the method is particularly cost-effective when primers can be reused. The author acknowledges some limitations of the method, such as the requirement for compatible buffers and tempera

In [52]:
abstract_list = abstract_list[0:10]

In [53]:
len(abstract_list)

10

In [54]:
from rouge_score import rouge_scorer

In [55]:
import time

# Start time
start_time = time.time()

#looping to go accross the entire data for evaluation

# Initialize BLEU and ROUGE scorers
bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
meteor_scores = []

rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])

# Summarize osmosis transcripts and evaluate
for i, note in enumerate(text_list):

    #prompt = prompt_template.format(
     #   context=note
    #)
    #cli_notes[0][i].append(note)
    #summary = get_model_response(prompt)[0][1]
    #gen_sum[0][i].append(summary)      ##########################update##########################

    # Evaluate BLEU score
    bleu_score = corpus_bleu([[abstract_list[i].split()]], [reference_summaries[i].split()])
    bleu_scores.append(bleu_score)

    # Evaluate ROUGE scores
    reference = abstract_list[i]
    hypothesis = reference_summaries[i]
    rouge_scores = rouge_scorer.score(reference, hypothesis)

    rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
    rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
    rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

     # Tokenize hypothesis and reference summaries
    hypothesis_tokens = word_tokenize(hypothesis)
    reference_tokens = word_tokenize(reference)

    # Evaluate METEOR score
    meteor_score_value = meteor_score.meteor_score([reference_tokens], hypothesis_tokens)
    meteor_scores.append(meteor_score_value)

    print("Article:")
    print(note)
    print("\nReference Summary:")
    print(abstract_list[i])
    print("\nGenerated Summary:")
    print(reference_summaries[i])
    print("\nBLEU Score:", bleu_score)
    print("ROUGE Scores:", rouge_scores)
    print("METEOR Score:", meteor_score_value)
    print("--------------------------------------------------")

    
end_time = time.time()

Article:
background with hundreds of enzymes commercially available today , restriction endonuclease treatment of insert and plasmid vector dna followed by ligation and transformation into competent e.
coli strains presents the standard cloning method in molecular biology.
given the advances in structural biology and the advent of synthetic biology, a strong demand exists to transfer and rearrange a large variety of dna fragments from different genetic sources in a directed manner.
a diverse catalogue of plasmid vectors is at hand for propagation in pro and eukaryotic cells, enabling heterologous protein expression in various host organisms.
frequently, suitable pairs of type ii restriction enzymes with unique recognition sites in the vector and insert dna fragments can be found, especially since the latter are easily produced via pcr.
in such a case, the pcr primers contain addon tails composed of the restriction endonuclease recognition sequence and additional nucleotides which ensur

Article:
background nongenotoxic carcinogens are difficult to screen for as the development of tumours only occurs after long term exposure and, since there is no damage to dna, they cannot be detected by conventional genotoxicity assays such as the ames test, in vitro clastogenesis assays or the mouse micronucleus test.
definitive characterization of nongenotoxic rat carcinogens requires the outcome of a two year bioassay.
there are earlier indicators of hepatocarcinogenic potential which include increased liver weight, evidence of cytotoxicity, enzyme induction and car agonist activity which are used to aid dose setting for long term bioassays and in estimating risk.
however, their predictive power is limited and complicated by the fact that nongenotoxic carcinogens represent a diverse range of mechanisms .
phenobarbital is a well characterized rodent nongenotoxic carcinogen which causes an increase in the incidence of liver tumors after long term exposure.
this follows a reversible 

Article:
background complex human genetic disease molecular biologists and geneticists alike now acknowledge that the most common human diseases with a genetic component are likely to have very complex etiologies, involving such complicating factors as locus heterogeneity, trait heterogeneity, and genegene interactions .
however, only a small fraction of the human genetics literature specifically reports on investigations of such complexity.
statistical geneticists continue primarily using traditional methodologies, such as linkage and association, which often detect but fail to replicate findings of main effect genes.
while undoubtedly many of the original positive results are falsepositives, true effects may not be replicated for many reasons, including population stratification and true differences in genetic etiology between study populations .
current statistical approaches to detecting heterogeneity, such as the admixture test , are neither sensitive nor powerful and can merely a

Article:
background senescence in plants is an intrinsic, genetically determined, natural developmental programme that operates at the end of leaf, fruit, or flower development .
it is characterized by the visible yellowing of leaves accompanied by the mobilization of leaf nutrients to the reproductive structures, and is a complex process involving changes of physiological, biochemical and gene expression regulated by endogenous and exogenous factors .
as senescence occurs at the ultimate stage in leaf development and precedes cell death , it has a crucial impact on agriculture, especially in crops where crop yield is enhanced by longer growth periods.
although leaf senescence is controlled mainly by developmental age, it can be modulated or triggered by adverse environmental factors such as temperature, high salinity, drought, submergence, ozone, constant darkness, nutrient deficiency, light and pathogen infection .
therefore, an understanding of leaf senescence mechanisms is importan

Article:
background ant colony algorithm belongs to the class of problemsolving strategies derived from nature.
dorigo, maniezzo, and colorni , using the traveling salesman problem as example, introduced the first ac algorithm.
with the further study in this area, ant colony algorithm is widely applied to the problems of quadratic assignment problem , the frequence assignment problem , the sequential ordering problem and some other npcomplete problems.
it demonstrates its superiority of solving complicated combinatorial optimization problems.
ac algorithms have been inspired by colonies of real ants, which deposit a chemical substance called pheromone on the ground.
this substance influences the choices they make: the more pheromone is on a particular path, the larger the probability is that an ant selects the path.
artificial ants in ac algorithms behave in a similar way.
thus, these behaviours of ant colony construct a positive feedback loop, and the pheromone is used to communicate 

Article:
background enterobacter sakazakii was defined as a new species in by farmer et al .
dnadna hybridization gave no clear generic assignment for e.
sakazakii as it was shown to be related to species in two different genera, enterobacter and citrobacter.
however the species was placed in enterobacter as it appeared phenotypically and genotypically closer to e.
cloacae than to c.
freundii, the type species of these genera.
in the original study fifteen biogroups of e.
sakazakii were described.
recently the existence of a sixteenth biogroup has been reported and a correlation between 16s rrna gene sequence analysis, which separated e.
sakazakii strains into several genetic groups, and biogroups has been demonstrated .
the existence of these divergent groups seems to support the suggestion of farmer et al that e.
sakazakii may harbour different species .
however previous studies were based on only partial 16s rrna gene sequence analysis, whereas for taxonomical purposes the complete 

In [56]:
time_taken = end_time - start_time
print(f"Time taken: {time_taken} seconds")

Time taken: 1.6936640739440918 seconds


In [57]:
# Calculate and print average BLEU score
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU Score:", avg_bleu_score)

# Calculate and print average ROUGE scores
avg_rouge1_score = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2_score = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL_score = sum(rougeL_scores) / len(rougeL_scores)
print("Average ROUGE-1 Score:", avg_rouge1_score)
print("Average ROUGE-2 Score:", avg_rouge2_score)
print("Average ROUGE-L Score:", avg_rougeL_score)

# Calculate and print average METEOR score
avg_meteor_score = sum(meteor_scores) / len(meteor_scores)
print("Average METEOR Score:", avg_meteor_score)

Average BLEU Score: 0.0030864729367520083
Average ROUGE-1 Score: 0.17726798032177762
Average ROUGE-2 Score: 0.02682861510532073
Average ROUGE-L Score: 0.09679499439036009
Average METEOR Score: 0.2120722100310643


In [58]:
#results

Average BLEU Score: 0.0029386505135258374

Average ROUGE-1 Score: 0.1682518712802705

Average ROUGE-2 Score: 0.02537887466792847

Average ROUGE-L Score: 0.09371105367290831

Average METEOR Score: 0.20889490173181802

summarize the text below results

Average BLEU Score: 0.0030864729367520083

Average ROUGE-1 Score: 0.17726798032177762

Average ROUGE-2 Score: 0.02682861510532073

Average ROUGE-L Score: 0.09679499439036009

Average METEOR Score: 0.2120722100310643

In [ ]:
#without chunking 

In [ ]:
text_list = text_list[0:10]
abstract_list = abstract_list[0:10]

In [59]:
print(len(text_list))
print(len(abstract_list))

10
10


In [69]:
#create a prompt template

template = (
    """
    {context}
    
    
    Please summarize the above text 
    """
)
prompt_template = PromptTemplate.from_template(template)
print(prompt_template.input_variables)

['context']


In [70]:
from rouge_score import rouge_scorer

In [71]:
import time

# Start time
start_time = time.time()

#looping to go accross the entire data for evaluation

# Initialize BLEU and ROUGE scorers
bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
meteor_scores = []

rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])

# Summarize osmosis transcripts and evaluate
for i, note in enumerate(text_list):

    prompt = prompt_template.format(
        context=note
    )
    #cli_notes[0][i].append(note)
    summary = get_model_response(prompt)[0][1]
    #gen_sum[0][i].append(summary)      ##########################update##########################

    # Evaluate BLEU score
    bleu_score = corpus_bleu([[abstract_list[i].split()]], [summary.split()])
    bleu_scores.append(bleu_score)

    # Evaluate ROUGE scores
    reference = abstract_list[i]
    hypothesis = summary
    rouge_scores = rouge_scorer.score(reference, hypothesis)

    rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
    rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
    rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

     # Tokenize hypothesis and reference summaries
    hypothesis_tokens = word_tokenize(hypothesis)
    reference_tokens = word_tokenize(reference)

    # Evaluate METEOR score
    meteor_score_value = meteor_score.meteor_score([reference_tokens], hypothesis_tokens)
    meteor_scores.append(meteor_score_value)

    #print("Article:")
    #print(note)
    print("\nReference Summary:")
    print(abstract_list[i])
    print("\nGenerated Summary:")
    print(summary)
    print("\nBLEU Score:", bleu_score)
    print("ROUGE Scores:", rouge_scores)
    print("METEOR Score:", meteor_score_value)
    print("--------------------------------------------------")

    
end_time = time.time()

/Users/ritvikkhandelwal/opt/anaconda3/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)



Reference Summary:
background dehydrins protect plant cells from desiccation damage during environmental stress, and also participate in host resistance to various pathogens.
in this study, we aimed to identify and characterize the dhn gene families from vitis vinifera and wild v.
yeshanensis, which is tolerant to both drought and cold, and moderately resistant to powdery mildew.
results four dhn genes were identified in both v.
vinifera and v.
yeshanensis, which shared a high sequence identity between the two species but little homology between the genes themselves.
these genes were designated dhn dhn dhn and dhn all four of the dhn proteins were highly hydrophilic and were predicted to be intrinsically disordered, but they differed in their isoelectric points, kinase selectivities and number of functional motifs.
also, the expression profiles of each gene differed appreciably from one another.
grapevine dhn was not expressed in vegetative tissues under normal growth conditions, but 


Reference Summary:
background to investigate antibiotic resistance, the occurrence and distribution of class and integrons in multidrug resistant staphylococcus aureus isolates from hospitals in tehran, iran.
the isolates were examined for susceptibility to antimicrobial agents.
the meca gene, class and integrons were detected by pcr.
integrase positive strains were further analysed for the presence of resistance gene cassettes using specific primers and were sequenced.
results among s.
aureus isolates, and strains were considered as multidrug resistant and meca positive, respectively.
class integrons and internal variable regions were found in and and class integrons and variable regions also in and of s.
aureus clinical isolates, respectively.
twelve distinct cassette arrays were found, containing genes encoding resistance to lactams, aminoglycosides, streptothricin, trimethoprim, chloramphenicol,a putative glucose dehydrogenase precursor and a protein with unknown function.
gene ca


Reference Summary:
background amur ide is an economically and ecologically important cyprinid species in northern asia.
the dali nor population living in the soda lake dali nor can adapt the extremely high alkalinity, providing us a valuable material to understand the adaptation mechanism against extreme environmental stress in teleost.
results in this study, we generated highthroughput rnaseq data from three tissues gill, liver and kidney of l.
waleckii living in the soda lake dali nor and the fresh water lake ganggeng nor, then performed parallel comparisons of three tissues.
our results showed that out of assembled transcript contigs, contigs had been assigned with a known function, corresponding to unique protein accessions.
we found and differentially expressed genes in the gill, kidney, and liver, respectively, of dali nor population compared to ganggeng nor population with fdr and foldchange further analysis revealed that wellknown functional categories of genes and signaling p


Reference Summary:
background nongenotoxic carcinogens are notoriously difficult to identify as they do not damage dna directly and have diverse modes of action, necessitating long term in vivo studies.
the early effects of the classic rodent nongenotoxic hepatocarcinogen phenobarbital have been investigated in the fisher rat using a combination of metabolomics and transcriptomics, to investige early stage mechanistic changes that are predictive of longer term pathology.
results liver and blood plasma were profiled across days, and multivariate statistics used to identify perturbed pathways.
both metabolomics and transcriptomics detected changes in the liver which were dose dependent, even after one day of exposure.
integration of the two datasets associated perturbations with specific pathways.
hepatic glycogen was decreased due to a decrease in synthesis, and plasma triglycerides were decreased due to an increase in fatty acid uptake by the liver.
hepatic succinate was increased and

In [72]:
time_taken = end_time - start_time
print(f"Time taken: {time_taken} seconds")

Time taken: 122.73916697502136 seconds


In [73]:
# Calculate and print average BLEU score
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU Score:", avg_bleu_score)

# Calculate and print average ROUGE scores
avg_rouge1_score = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2_score = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL_score = sum(rougeL_scores) / len(rougeL_scores)
print("Average ROUGE-1 Score:", avg_rouge1_score)
print("Average ROUGE-2 Score:", avg_rouge2_score)
print("Average ROUGE-L Score:", avg_rougeL_score)

# Calculate and print average METEOR score
avg_meteor_score = sum(meteor_scores) / len(meteor_scores)
print("Average METEOR Score:", avg_meteor_score)

Average BLEU Score: 0.004938957431836572
Average ROUGE-1 Score: 0.20363664469704604
Average ROUGE-2 Score: 0.022824041824316157
Average ROUGE-L Score: 0.12106091519538195
Average METEOR Score: 0.1196723556753537


In [67]:
#results

Average BLEU Score: 0.004938957431836572
Average ROUGE-1 Score: 0.20363664469704604
Average ROUGE-2 Score: 0.022824041824316157
Average ROUGE-L Score: 0.12106091519538195
Average METEOR Score: 0.11967235567535371

#Please summarize the text below 

Average BLEU Score: 0.003357773428794034
Average ROUGE-1 Score: 0.2057030552431991
Average ROUGE-2 Score: 0.015498985074409746
Average ROUGE-L Score: 0.11302348578717691
Average METEOR Score: 0.16548278167980424